In [1]:
%config IPCompleter.greedy=True

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import json
import pprint

# 构建简单的模型

## 序列模型
最常见模型类型是层的堆叠`tf.keras.Sequential`模型，构建简单的全连接网络（即多层感知器）

In [3]:
# 创建模型
mode=keras.Sequential()

## 配置层
使用`tf.keras.layers`方法创建层，重要参数：
- `activation`：设置层的激活函数
- `kernel_initizlizer`和`bias_initializer`：创建层权重的初始化方案
- `kernel_regularizer`和`bias_regularizer`：应用层权重的正则化方案

In [4]:
# 配置层
mode.add(keras.layers.Dense(64,activation='relu'))
mode.add(keras.layers.Dense(64,activation='relu'))
mode.add(keras.layers.Dense(10,activation='softmax'))

`compile`方法配置模型的__学习流程__，三个重要参数
- `optimizer`:此对象指定训练过程
- `loss`:在优化期间最小化的函数
- `metrics`:用于监控训练

In [5]:
# 配置模型的学习流程
# 方式一：
mode.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
# # 方法二：
# mode.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
#              loss=tf.keras.losses.categorical_crossentropy,
#              metrics=[tf.keras.metrics.categorical_crossentropy])


In [6]:
def random_data():
    data = np.random.random((1000,32))
    labels=np.random.random((1000,10))
    return data,labels

## 训练
`fit`方法的三个重要参数：
- `epochs`:以周期为单位进行训练
- `batch_size`:当传递np数据时，模型将数据分成较小的批次，并在训练期间迭代这些批次
- `validation_data`:对模型进行原型设计时，需要轻松监控该模型在某些验证数据上达到的效果

### np数据

拟合数据

In [7]:
# 获取np数据
data,labels=random_data()
# 训练模型，使模型与训练数据拟合
mode.fit(data,labels,epochs=10,batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s 313us/step - loss: 11.5382 - acc: 0.1100
Epoch 2/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.4855 - acc: 0.1190
Epoch 3/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.4778 - acc: 0.1190
Epoch 4/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.4726 - acc: 0.1290
Epoch 5/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.4682 - acc: 0.1310
Epoch 6/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.4643 - acc: 0.1260
Epoch 7/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.4595 - acc: 0.1320
Epoch 8/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.4558 - acc: 0.1260
Epoch 9/10
1000/1000 [==============================] - 0s 25us/step - loss: 11.4512 - acc: 0.1500
Epoch 10/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.4482 - acc: 0.1500


拟合数据并进行验证

In [8]:
data,labels=random_data()
vdata,vlabels=random_data()

mode.fit(data,labels,epochs=10,batch_size=32,validation_data=(vdata,vlabels))

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 55us/step - loss: 11.5522 - acc: 0.0790 - val_loss: 11.4582 - val_acc: 0.0960
Epoch 2/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.5370 - acc: 0.1130 - val_loss: 11.4560 - val_acc: 0.0970
Epoch 3/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.5316 - acc: 0.1280 - val_loss: 11.4563 - val_acc: 0.0850
Epoch 4/10
1000/1000 [==============================] - 0s 30us/step - loss: 11.5257 - acc: 0.1200 - val_loss: 11.4580 - val_acc: 0.0990
Epoch 5/10
1000/1000 [==============================] - 0s 36us/step - loss: 11.5214 - acc: 0.1290 - val_loss: 11.4587 - val_acc: 0.0880
Epoch 6/10
1000/1000 [==============================] - 0s 35us/step - loss: 11.5178 - acc: 0.1390 - val_loss: 11.4618 - val_acc: 0.0960
Epoch 7/10
1000/1000 [==============================] - 0s 30us/step - loss: 11.5132 - acc: 0.1470 - val_loss: 11.4600 - val_acc: 

### 输入数据集

使用dataset数据

In [9]:
dataset=tf.data.Dataset.from_tensor_slices((data,labels))
dataset=dataset.batch(32)
dataset=dataset.repeat()

`fit`方法使用`steps_per_epoch`参数(表示模型在进入下一个周期之前运行的训练步数)。由于`Dataset`会生成批次数据，因此该代码段不需要`batch_size`

拟合数据

In [10]:
mode.fit(dataset,epochs=10,steps_per_epoch=30)

Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4995 - acc: 0.1604
Epoch 2/10
30/30 [==============================] - 0s 931us/step - loss: 11.5377 - acc: 0.1594
Epoch 3/10
30/30 [==============================] - 0s 964us/step - loss: 11.5063 - acc: 0.1677
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5286 - acc: 0.1625
Epoch 5/10
30/30 [==============================] - 0s 964us/step - loss: 11.5099 - acc: 0.1646
Epoch 6/10
30/30 [==============================] - 0s 931us/step - loss: 11.4999 - acc: 0.1573
Epoch 7/10
30/30 [==============================] - 0s 831us/step - loss: 11.5390 - acc: 0.1740
Epoch 8/10
30/30 [==============================] - 0s 798us/step - loss: 11.4952 - acc: 0.1771
Epoch 9/10
30/30 [==============================] - ETA: 0s - loss: 11.6524 - acc: 0.18 - 0s 931us/step - loss: 11.5156 - acc: 0.1719
Epoch 10/10
30/30 [==============================] - 0s 831us/step - loss: 11.5058 - acc: 0.1823


使用dataset数据与验证数据

In [11]:
dataset=tf.data.Dataset.from_tensor_slices((data,labels))
dataset=dataset.batch(32).repeat()

vdataset=tf.data.Dataset.from_tensor_slices((data,labels))
vdataset=vdataset.batch(32).repeat()

拟合数据并进行验证

In [12]:
mode.fit(dataset,epochs=10,steps_per_epoch=30,validation_data=vdataset,validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 5ms/step - loss: 11.4581 - acc: 0.1948 - val_loss: 11.3925 - val_acc: 0.1562
Epoch 2/10
30/30 [==============================] - 0s 698us/step - loss: 11.4940 - acc: 0.1969 - val_loss: 11.6574 - val_acc: 0.2396
Epoch 3/10
30/30 [==============================] - 0s 931us/step - loss: 11.4604 - acc: 0.1958 - val_loss: 11.4830 - val_acc: 0.1771
Epoch 4/10
30/30 [==============================] - 0s 863us/step - loss: 11.4838 - acc: 0.1917 - val_loss: 11.1180 - val_acc: 0.2500
Epoch 5/10
30/30 [==============================] - 0s 831us/step - loss: 11.4639 - acc: 0.2052 - val_loss: 11.0720 - val_acc: 0.2083
Epoch 6/10
30/30 [==============================] - 0s 731us/step - loss: 11.4536 - acc: 0.2031 - val_loss: 11.7156 - val_acc: 0.1771
Epoch 7/10
30/30 [==============================] - 0s 865us/step - loss: 11.4932 - acc: 0.1958 - val_loss: 11.2724 - val_acc: 0.2604
Epoch 8/10
30/30 [==============================] - 0s 698us/ste

## 评估和预测

### 评估

评估所提供数据的推理模式损失和指标

In [13]:
data,labels=random_data()
# np数据评估
mode.evaluate(data,labels,batch_size=32)
# 数据集评估
mode.evaluate(dataset,steps=30)

30/30 [==============================] - 0s 1ms/step


[11.40997527440389, 0.23333333333333334]

### 预测

在提供数据的推理中预测最后一层的输出

In [14]:
data,_=random_data()
result=mode.predict(data,batch_size=32)
print(result.shape)

(1000, 10)


In [15]:
result[:3]

array([[0.10691622, 0.09844066, 0.09566999, 0.07328409, 0.14637627,
        0.0934908 , 0.12641561, 0.0985145 , 0.08315478, 0.0777372 ],
       [0.0827874 , 0.06978317, 0.09850818, 0.10871132, 0.12982221,
        0.1014108 , 0.0881303 , 0.13951005, 0.09126016, 0.09007637],
       [0.08995933, 0.10691826, 0.10005227, 0.10218644, 0.09650246,
        0.11524831, 0.09682024, 0.10151553, 0.10202208, 0.08877509]],
      dtype=float32)

# 构建高级模型

## 函数式API
`tf.kerasa.Sequential`模型是层的简单堆叠，无法表示任意模型

用__Keras函数式API__可以构建复杂的模型拓扑，例如：
- 多输入模型
- 多输出模型
- 具有共享层模型（同一层被调用多次）
- 具有非序列数据流的模型（例如，剩余连接）

使用函数式API构建的模型具有以下特征：
- 层实例可调用并返回张量
- 输入张量和输出张量用于定义`tf.keras.Model`实例
- 此模型的训练方式和`Sequential`模型一样

使用函数式API构建简单的全连接网络

In [16]:
inputs=tf.keras.Input(shape=(32,))

# 层实例调用并返回张量
x=keras.layers.Dense(64,activation='relu')(inputs)
x=keras.layers.Dense(64,activation='relu')(x)
predictions = keras.layers.Dense(10,activation='softmax')(x)

在给定输入和输出的情况下实例化模型

In [17]:
model = keras.Model(inputs=inputs,outputs=predictions)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(data,labels,batch_size=32,epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 175us/step - loss: 11.6860 - acc: 0.1070
Epoch 2/5
1000/1000 [==============================] - 0s 24us/step - loss: 11.6295 - acc: 0.1120
Epoch 3/5
1000/1000 [==============================] - 0s 22us/step - loss: 11.6059 - acc: 0.1120
Epoch 4/5
1000/1000 [==============================] - 0s 24us/step - loss: 11.5868 - acc: 0.1100
Epoch 5/5
1000/1000 [==============================] - 0s 24us/step - loss: 11.5743 - acc: 0.1160


## 模型子类化
通过`tf.keras.Model`进行子类化定义自己的__前向传播__来构建自定义的模型
- 在`__init__`方法中创建层
- 在`call`方法中定义前向传播

使用自定义前身传播进行子类化的`tf.keras.Model`：

In [18]:
class MyModel(tf.keras.Model):
    def __init__(self,num_class=10):
        super(MyModel,self).__init__(name='my_model')
        self.num_class=num_class
        self.dense_1=keras.layers.Dense(32,activation='relu')
        self.dense_2=keras.layers.Dense(num_class,activation='sigmoid')
        
    def call(self,inputs):
        '''定义前向传播'''
        x=self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self,input_shape):
        '''使用子类化的模型，则需要覆盖此函数'''
        shape=tf.TensorShape(input_shape).as_list()
        shape[-1]=self.num_class
        return tf.TensorShape(shape)

实例化新模型类：

In [19]:
# 创建自定义的模型
model = MyModel(10)

# 配置模型的学习流程
model.compile(optimizer=tf.train.AdamOptimizer(0.1),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_crossentropy])

# 拟合数据
mode.fit(data,labels,batch_size=32,epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 142us/step - loss: 11.6137 - acc: 0.0840
Epoch 2/5
1000/1000 [==============================] - 0s 25us/step - loss: 11.5772 - acc: 0.1030
Epoch 3/5
1000/1000 [==============================] - 0s 23us/step - loss: 11.5675 - acc: 0.1210
Epoch 4/5
1000/1000 [==============================] - 0s 23us/step - loss: 11.5620 - acc: 0.1130
Epoch 5/5
1000/1000 [==============================] - 0s 23us/step - loss: 11.5561 - acc: 0.1210


## 自定义层
通过`tf.keras.layers.Layer`进行子类化并实现以下方法来创建自定义层：
- `build`：创建层的权重
- `call`：定义前向传播
- `computer_output_shape`：指定在给定输入形状的情况下如何计算层的输出形状
- 可以通过实现`get_config`方法和`from_config`类方法序列化层

使用核矩阵实现输入`matmul`的自定义层示例：

In [20]:
class MyLayer(keras.layers.Layer):
    def __init__(self,output_dim,**kwargs):
        self.output_dim=output_dim
        super(MyLayer,self).__init__(**kwargs)
    
    def build(self,input_shape):
        shape=tf.TensorShape((input_shape[1],self.output_dim))
        
#         为这一层创建一个可训练的权重变量
        self.kernel=self.add_weight(name='kernel',shape=shape,initializer='uniform',trainable=True)
        
#         最后一定要调用这个
        super(MyLayer,self).build(input_shape)
    
    def call(self,inputs):
        return tf.matmul(inputs,self.kernel)
    
    def complete_output_shape(self,input_shape):
        shape=tf.TensorShape(input_shape).as_list()
        shape[-1]=self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config=super(MyLayer,self).get_config()
        base_config['output_dim']=self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls,config):
        return cls(**config)
    

使用自定义层创建模型：

In [21]:
# 创建序列模型
model=tf.keras.Sequential([MyLayer(10),keras.layers.Activation('softmax')])

# 配置模型的学习流程
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss=keras.losses.categorical_crossentropy,
             metrics=[keras.metrics.categorical_accuracy])
# 拟合数据
model.fit(data,labels,batch_size=32,epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 140us/step - loss: 12.6504 - categorical_accuracy: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 20us/step - loss: 12.3413 - categorical_accuracy: 0.1090
Epoch 3/5
1000/1000 [==============================] - 0s 19us/step - loss: 12.1103 - categorical_accuracy: 0.1120
Epoch 4/5
1000/1000 [==============================] - 0s 19us/step - loss: 12.0407 - categorical_accuracy: 0.1140
Epoch 5/5
1000/1000 [==============================] - 0s 19us/step - loss: 12.0077 - categorical_accuracy: 0.1130


## 回调
回调是传递给模型的__对象__，用于在训练期间自定义该模型并扩展其行为
- `tf.keras.callbacks.ModelCheckpoint`：定期保存模型的检查点
- `tf.keras.callbacks.LearningRateScheduler`：动态更改学习速率
- `tf.keras.callbacks.EarlyStoppint`：在验证效果不再改进时中断训练
- `tf.keras.callbacks.TensorBoard`：监控模型的行为

In [22]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(vdata, vlabels))

Train on 1000 samples, validate on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 62us/step - loss: 11.9804 - categorical_accuracy: 0.1140 - val_loss: 11.8559 - val_categorical_accuracy: 0.1080
Epoch 2/5
1000/1000 [==============================] - 0s 27us/step - loss: 11.9524 - categorical_accuracy: 0.1080 - val_loss: 11.8290 - val_categorical_accuracy: 0.1100
Epoch 3/5
1000/1000 [==============================] - 0s 27us/step - loss: 11.9264 - categorical_accuracy: 0.1110 - val_loss: 11.8060 - val_categorical_accuracy: 0.1040
Epoch 4/5
1000/1000 [==============================] - 0s 29us/step - loss: 11.9020 - categorical_accuracy: 0.1190 - val_loss: 11.7838 - val_categorical_accuracy: 0.1040
Epoch 5/5
1000/1000 [==============================] - 0s 28us/step - loss: 11.8778 - categorical_accuracy: 0.1170 - val_loss: 11.7632 - val_categorical_accuracy: 0.1070


# 保存和恢复

## 仅保存权重
使用`tf.keras.Model.save_weights`保存并加载模型的权重

### 检查点文件格式保存权重

In [23]:
model.save_weights('./weights/my_model')

In [24]:
model.load_weights('./weights/my_model')

### HDF5格式保存权重

In [25]:
model.save_weights('my_model.h5', save_format='h5')

In [26]:
model.load_weights('my_model.h5')

## 仅保存配置
可保存模型的配置，对模型架构进行序列化

即使没有定义原始模型的代码，该配置也可以重新创建并初始化相同的模型

__子类化模型不可序列化，因为它们的架构由`call`方法中的Python代码定义__

### json格式序列化

In [27]:
json_str=model.to_json()

In [28]:
# 格式化json_str
pprint.pprint(json.loads(json_str))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': [{'class_name': 'MyLayer',
             'config': {'batch_input_shape': [None, 32],
                        'dtype': 'float32',
                        'name': 'my_layer',
                        'output_dim': 10,
                        'trainable': True}},
            {'class_name': 'Activation',
             'config': {'activation': 'softmax',
                        'dtype': 'float32',
                        'name': 'activation',
                        'trainable': True}}],
 'keras_version': '2.1.6-tf'}


In [29]:
fresh_model=tf.keras.models.model_from_json(json_str)

ValueError: Unknown layer: MyLayer

### YAML格式序列化

In [ ]:
yaml_str=model.to_yaml()

In [ ]:
fresh_model=tf.keras.models.model_from_yaml(yaml_str)

## 保存整个模型
将整个模型保存到一个文件中，包括__权重值、模型配置、优化器配置__

便可以设置检查点并稍后从完全相同的状态继续训练，无需访问原始代码

In [ ]:
model.save('my_model.h5')

In [ ]:
model=tf.keras.models.load_model('my_model.h5')